In [9]:
# !pip3 install nltk
# !pip3 install wordcloud
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from IPython import display
import time
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


In [10]:
spark = SparkSession.builder\
        .master("local[*]")\
        .appName("SparkStreaming")\
        .getOrCreate()

22/11/07 22:51:44 WARN Utils: Your hostname, Eduardos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.3.5 instead (on interface en0)
22/11/07 22:51:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/07 22:51:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/07 22:51:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
def trata_tweets(df):
    words = df\
        .select(f.explode(f.split(f.lower("_c0")," ")).alias("word"))\
        .withColumn("word", f.regexp_replace("word", r'http\S+', ""))\
        .withColumn("word", f.regexp_replace("word", r'@\w+', ""))\
        .withColumn("word", f.regexp_replace("word", 'rt', ""))\
        .na.replace("",None) \
        .na.drop()
    return words
        

In [ ]:
stops = stopwords.words("portuguese")
stops.append ("futebol")
plt.figure(figsize=(20,10))

while True:
    try:
        words = spark.read.csv("./csv", encoding="utf-8")
        words = trata_tweets(words)
        rows = words.collect()
        all_words = ""
        for row in rows:
            all_words = all_words + " " + row["word"]
        wordcloud = WordCloud(  stopwords=stops,
                                background_color="black",
                                width=1920,
                                height=1080,
                                max_words=100).generate(all_words)
        plt.cla()
        plt.axis("off")
        plt.imshow(wordcloud)
        display.display(plt.gcf())
        display.clear_output(wait=True)
        time.sleep(5)
    except KeyboardInterrupt:
        break
